In [1]:
from __future__ import print_function
import numpy as np
import pandas as pd
from datetime import datetime, timedelta
import time

In [2]:
## Downloading the crime data -- very large, don't run again

!wget "https://data.cityofnewyork.us/api/views/qgea-i56i/rows.csv?accessType=DOWNLOAD"

In [3]:
# Reading in the crime data

crimeData = pd.read_csv("rows.csv_accessType=DOWNLOAD")

/usr/local/Anaconda3-5.0.0-Linux-x86_64/envs/PUI2016_Python3/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (17) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
# Looking through the columns

crimeData.columns

Index(['CMPLNT_NUM', 'CMPLNT_FR_DT', 'CMPLNT_FR_TM', 'CMPLNT_TO_DT',
       'CMPLNT_TO_TM', 'RPT_DT', 'KY_CD', 'OFNS_DESC', 'PD_CD', 'PD_DESC',
       'CRM_ATPT_CPTD_CD', 'LAW_CAT_CD', 'JURIS_DESC', 'BORO_NM',
       'ADDR_PCT_CD', 'LOC_OF_OCCUR_DESC', 'PREM_TYP_DESC', 'PARKS_NM',
       'HADEVELOPT', 'X_COORD_CD', 'Y_COORD_CD', 'Latitude', 'Longitude',
       'Lat_Lon'],
      dtype='object')

In [5]:
# Filtering down to complaint dates between 01/01/2006 - 12/31/2017

crimeData['year'] = crimeData['CMPLNT_FR_DT'].str[-4:].fillna(0).astype(int)
crimeData['month'] = crimeData['CMPLNT_FR_DT'].str[:2].fillna(0).astype(int)
crimeData['date'] = crimeData['CMPLNT_FR_DT'].str[3:5].fillna(0).astype(int)
crimeData['hour'] = crimeData['CMPLNT_FR_TM'].str[:2].fillna(0).astype(int)
crime = crimeData[((crimeData['year'] > 2005) & (crimeData['hour'] < 24))]

# Creating datetime column

#crime['datetime'] = pd.to_datetime((crime['CMPLNT_FR_DT'] + ' ' + crime['CMPLNT_FR_TM']), format='%m/%d/%Y %H:%M:%S')

In [6]:
crime.head()

,CMPLNT_NUM,CMPLNT_FR_DT,CMPLNT_FR_TM,CMPLNT_TO_DT,CMPLNT_TO_TM,RPT_DT,KY_CD,OFNS_DESC,PD_CD,PD_DESC,...,HADEVELOPT,X_COORD_CD,Y_COORD_CD,Latitude,Longitude,Lat_Lon,year,month,date,hour
0,101109527,12/31/2015,23:45:00,NaN,NaN,12/31/2015,113,FORGERY,729.0,"FORGERY,ETC.,UNCLASSIFIED-FELO",...,NaN,1007314.0,241257.0,40.828848,-73.916661,"(40.828848333, -73.916661142)",2015,12,31,23
1,153401121,12/31/2015,23:36:00,NaN,NaN,12/31/2015,101,MURDER & NON-NEGL. MANSLAUGHTER,NaN,NaN,...,NaN,1043991.0,193406.0,40.697338,-73.784557,"(40.697338138, -73.784556739)",2015,12,31,23
2,569369778,12/31/2015,23:30:00,NaN,NaN,12/31/2015,117,DANGEROUS DRUGS,503.0,"CONTROLLED SUBSTANCE,INTENT TO",...,NaN,999463.0,231690.0,40.802607,-73.945052,"(40.802606608, -73.945051911)",2015,12,31,23
3,968417082,12/31/2015,23:30:00,NaN,NaN,12/31/2015,344,ASSAULT 3 & RELATED OFFENSES,101.0,ASSAULT 3,...,NaN,1060183.0,177862.0,40.654549,-73.726339,"(40.654549444, -73.726338791)",2015,12,31,23
4,641637920,12/31/2015,23:25:00,12/31/2015,23:30:00,12/31/2015,344,ASSAULT 3 & RELATED OFFENSES,101.0,ASSAULT 3,...,NaN,987606.0,208148.0,40.738002,-73.987891,"(40.7380024, -73.98789129)",2015,12,31,23


In [7]:
# reducing 

sevenFelonies = ['ROBBERY', 'FELONY ASSAULT', 'MURDER & NON-NEGL. MANSLAUGHTER', 'GRAND LARCENY', 'BURGLARY',
                 'GRAND LARCENY OF MOTOR VEHICLE', 'RAPE']

dfSeven = crime.loc[crime['OFNS_DESC'].isin(sevenFelonies)].reset_index()

In [8]:
dfSeven.head()

,index,CMPLNT_NUM,CMPLNT_FR_DT,CMPLNT_FR_TM,CMPLNT_TO_DT,CMPLNT_TO_TM,RPT_DT,KY_CD,OFNS_DESC,PD_CD,...,HADEVELOPT,X_COORD_CD,Y_COORD_CD,Latitude,Longitude,Lat_Lon,year,month,date,hour
0,1,153401121,12/31/2015,23:36:00,NaN,NaN,12/31/2015,101,MURDER & NON-NEGL. MANSLAUGHTER,NaN,...,NaN,1043991.0,193406.0,40.697338,-73.784557,"(40.697338138, -73.784556739)",2015,12,31,23
1,5,365661343,12/31/2015,23:18:00,12/31/2015,23:25:00,12/31/2015,106,FELONY ASSAULT,109.0,...,NaN,996149.0,181562.0,40.665023,-73.957111,"(40.665022689, -73.957110763)",2015,12,31,23
2,12,898496564,12/31/2015,23:00:00,NaN,NaN,12/31/2015,109,GRAND LARCENY,406.0,...,NaN,987215.0,215403.0,40.757916,-73.989299,"(40.757915693, -73.98929902)",2015,12,31,23
3,13,566081066,12/31/2015,23:00:00,NaN,NaN,12/31/2015,109,GRAND LARCENY,415.0,...,NaN,987215.0,215403.0,40.757916,-73.989299,"(40.757915693, -73.98929902)",2015,12,31,23
4,15,715942154,12/31/2015,23:00:00,12/31/2015,23:05:00,12/31/2015,105,ROBBERY,386.0,...,NaN,1044662.0,197327.0,40.708096,-73.782102,"(40.708095777, -73.78210174)",2015,12,31,23


In [9]:
# dropping unnecessary columns

dfSevenDropped = dfSeven.drop(['CMPLNT_NUM', 'CMPLNT_FR_DT', 'CMPLNT_FR_TM', 'CMPLNT_TO_DT',
       'CMPLNT_TO_TM', 'RPT_DT', 'KY_CD', 'PD_CD', 'PD_DESC',
       'CRM_ATPT_CPTD_CD', 'LAW_CAT_CD', 'JURIS_DESC', 'LOC_OF_OCCUR_DESC',
       'PREM_TYP_DESC', 'PARKS_NM','HADEVELOPT', 'X_COORD_CD',
       'Y_COORD_CD', 'index', 'hour'], axis=1)

In [10]:
# cleaning the dataframe for sake of ease

crimeVisClean1 = dfSevenDropped.replace(to_replace='MURDER & NON-NEGL. MANSLAUGHTER', value='MURDER')
crimeVisClean2 = crimeVisClean1.replace(to_replace='GRAND LARCENY OF MOTOR VEHICLE', value='GRAND THEFT AUTO')
crimeVis = crimeVisClean2.rename(columns={"OFNS_DESC": "crimetype", 
                                  "ADDR_PCT_CD": "precinct",
                                  "BORO_NM": "borough"})

In [11]:
crimeVis.head()

,crimetype,borough,precinct,Latitude,Longitude,Lat_Lon,year,month,date
0,MURDER,QUEENS,103.0,40.697338,-73.784557,"(40.697338138, -73.784556739)",2015,12,31
1,FELONY ASSAULT,BROOKLYN,71.0,40.665023,-73.957111,"(40.665022689, -73.957110763)",2015,12,31
2,GRAND LARCENY,MANHATTAN,14.0,40.757916,-73.989299,"(40.757915693, -73.98929902)",2015,12,31
3,GRAND LARCENY,MANHATTAN,14.0,40.757916,-73.989299,"(40.757915693, -73.98929902)",2015,12,31
4,ROBBERY,QUEENS,103.0,40.708096,-73.782102,"(40.708095777, -73.78210174)",2015,12,31


In [12]:
# Dropping crimes without location data

crimeVis = crimeVis.dropna(axis = 0, subset=['Lat_Lon'])

In [13]:
crimeVis.crimetype.unique()

array(['MURDER', 'FELONY ASSAULT', 'GRAND LARCENY', 'ROBBERY', 'BURGLARY',
       'GRAND THEFT AUTO'], dtype=object)

#### We realized that all rape observations have been wiped of specific lat-lon information, likely for privacy purposes. Because we dropped all crimes without lat-lon, this removed rape from the dataframe altogether

In [14]:
# limit dataframe so that it won't be too large for Carto

crimeVisFinal = crimeVis[crimeVis['year'] >= 2012]

In [15]:
# export dataframe to csv for upload to Carto

crimeVisFinal.to_csv('crimeVisFinal.csv', index=False)